## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

In [2]:
from google.colab import files
uploaded = files.upload()

Saving charity_data.csv to charity_data.csv


In [3]:
import io
charity_df = pd.read_csv(io.BytesIO(uploaded['charity_data.csv']))

In [4]:
charity_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [5]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
charity_df = charity_df.drop(columns=["NAME", "EIN", "AFFILIATION"])
charity_df

,APPLICATION_TYPE,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...
34294,T4,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,C3000,ProductDev,Association,1,0,N,5000,1


In [6]:
# Determine the number of unique values in each column.
charity_df.nunique(axis=0)

APPLICATION_TYPE            17
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [7]:
# Look at APPLICATION_TYPE value counts for binning
charity_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [8]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace =['T8','T7','T10','T9','T13','T12','T2','T25','T14','T29','T15','T17']

# Replace in dataframe
for app in application_types_to_replace:
    charity_df['APPLICATION_TYPE'] = charity_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
charity_df['APPLICATION_TYPE'].value_counts()

T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: APPLICATION_TYPE, dtype: int64

In [9]:
# Look at CLASSIFICATION value counts for binning
charity_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [10]:
# You may find it helpful to look at CLASSIFICATION value counts >1
charity_df = charity_df[charity_df.groupby("CLASSIFICATION")['CLASSIFICATION'].transform('size') > 10]
charity_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C2300       32
C7200       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
Name: CLASSIFICATION, dtype: int64

In [12]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
values_list = pd.unique(charity_df['CLASSIFICATION'])
values_list
classifications_to_replace = ['C7200',
       'C1700', 'C4000', 'C7100', 'C2800', 'C6000', 'C2100', 'C5000',
       'C7120', 'C1800', 'C1400', 'C1270', 'C2300', 'C8200', 'C1500',
       'C1300', 'C1230', 'C1280', 'C1240', 'C1250', 'C8000']

# Replace in dataframe
for cls in classifications_to_replace:
    charity_df['CLASSIFICATION'] = charity_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
charity_df['CLASSIFICATION'].value_counts()

<ipython-input-12-7b7050eb152b>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  charity_df['CLASSIFICATION'] = charity_df['CLASSIFICATION'].replace(cls,"Other")


C1000    17326
C2000     6074
C1200     4837
Other     3145
C3000     1918
C7000      777
C2700      104
Name: CLASSIFICATION, dtype: int64

In [13]:
# Convert categorical data to numeric with `pd.get_dummies`
charity_df= pd.get_dummies(charity_df)
charity_df

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,CLASSIFICATION_C1000,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,1,0,0,0,1,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,1,0
34295,1,5000,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34296,1,5000,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34297,1,5000,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0


In [14]:
# Split our preprocessed data into our features and target arrays
# Define the X (features) and y (target) sets
y = charity_df["IS_SUCCESSFUL"].values
X = charity_df.drop("IS_SUCCESSFUL", axis="columns").values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=50)

In [15]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [16]:
# # Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
# nn = tf.keras.models.Sequential()

# # First hidden layer
# nn.add(tf.keras.layers.Dense(units=80, activation="relu",input_dim=2))

# # Second hidden layer
# nn.add(tf.keras.layers.Dense(units=30, activation="relu"))

# # Output layer
# nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# # Check the structure of the model
# nn.summary()
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  20
hidden_nodes_layer2 = 10

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 20)                720       
                                                                 
 dense_1 (Dense)             (None, 10)                210       
                                                                 
 dense_2 (Dense)             (None, 1)                 11        
                                                                 
Total params: 941
Trainable params: 941
Non-trainable params: 0
_________________________________________________________________


In [17]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [18]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
802/802 [==============================] - 3s 2ms/step - loss: 0.6480 - accuracy: 0.6098
Epoch 2/100
802/802 [==============================] - 2s 2ms/step - loss: 0.6278 - accuracy: 0.6362
Epoch 3/100
802/802 [==============================] - 2s 2ms/step - loss: 0.6253 - accuracy: 0.6385
Epoch 4/100
802/802 [==============================] - 2s 2ms/step - loss: 0.6244 - accuracy: 0.6392
Epoch 5/100
802/802 [==============================] - 2s 3ms/step - loss: 0.6234 - accuracy: 0.6399
Epoch 6/100
802/802 [==============================] - 2s 3ms/step - loss: 0.6228 - accuracy: 0.6403
Epoch 7/100
802/802 [==============================] - 2s 2ms/step - loss: 0.6223 - accuracy: 0.6408
Epoch 8/100
802/802 [==============================] - 2s 2ms/step - loss: 0.6218 - accuracy: 0.6405
Epoch 9/100
802/802 [==============================] - 2s 2ms/step - loss: 0.6220 - accuracy: 0.6413
Epoch 10/100
802/802 [==============================] - 2s 2ms/step - loss: 0.6213 - accura

In [19]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.6302 - accuracy: 0.6451 - 635ms/epoch - 2ms/step
Loss: 0.6302006840705872, Accuracy: 0.6450971364974976


In [20]:
# Export our model to HDF5 file
from google.colab import drive
drive.mount('/content/gdrive')
nn.save("/content/gdrive/MyDrive/AlphabetSoupCharity1.h5")

Mounted at /content/gdrive


In [22]:
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 70
hidden_nodes_layer2 = 

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="relu"))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 20)                720       
                                                                 
 dense_7 (Dense)             (None, 10)                210       
                                                                 
 dense_8 (Dense)             (None, 1)                 11        
                                                                 
Total params: 941
Trainable params: 941
Non-trainable params: 0
_________________________________________________________________


In [50]:
len(X_train[0])

35

In [23]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [24]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=150)

Epoch 1/150
802/802 [==============================] - 4s 3ms/step - loss: 1.2637 - accuracy: 0.5848
Epoch 2/150
802/802 [==============================] - 2s 2ms/step - loss: 0.9861 - accuracy: 0.6102
Epoch 3/150
802/802 [==============================] - 2s 2ms/step - loss: 0.9309 - accuracy: 0.6178
Epoch 4/150
802/802 [==============================] - 2s 2ms/step - loss: 0.8937 - accuracy: 0.6241
Epoch 5/150
802/802 [==============================] - 2s 2ms/step - loss: 0.8769 - accuracy: 0.6272
Epoch 6/150
802/802 [==============================] - 2s 2ms/step - loss: 0.8117 - accuracy: 0.6305
Epoch 7/150
802/802 [==============================] - 2s 2ms/step - loss: 1.0063 - accuracy: 0.6028
Epoch 8/150
802/802 [==============================] - 2s 3ms/step - loss: 0.8948 - accuracy: 0.6134
Epoch 9/150
802/802 [==============================] - 2s 3ms/step - loss: 0.7996 - accuracy: 0.6307
Epoch 10/150
802/802 [==============================] - 2s 2ms/step - loss: 0.7094 - accura

In [25]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.6782 - accuracy: 0.6399 - 1s/epoch - 4ms/step
Loss: 0.6782336235046387, Accuracy: 0.6399484872817993


In [43]:
# Export our model to HDF5 file
nn.save("/content/gdrive/MyDrive/AlphabetSoupCharity2.h5")

In [52]:
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 60


nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_32 (Dense)            (None, 80)                2880      
                                                                 
 dense_33 (Dense)            (None, 60)                4860      
                                                                 
 dense_34 (Dense)            (None, 1)                 61        
                                                                 
Total params: 7,801
Trainable params: 7,801
Non-trainable params: 0
_________________________________________________________________


In [53]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [54]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=120)

Epoch 1/120
802/802 [==============================] - 3s 3ms/step - loss: 0.6354 - accuracy: 0.6286
Epoch 2/120
802/802 [==============================] - 3s 3ms/step - loss: 0.6284 - accuracy: 0.6355
Epoch 3/120
802/802 [==============================] - 2s 2ms/step - loss: 0.6253 - accuracy: 0.6391
Epoch 4/120
802/802 [==============================] - 2s 2ms/step - loss: 0.6239 - accuracy: 0.6380
Epoch 5/120
802/802 [==============================] - 2s 3ms/step - loss: 0.6230 - accuracy: 0.6412
Epoch 6/120
802/802 [==============================] - 2s 3ms/step - loss: 0.6220 - accuracy: 0.6418
Epoch 7/120
802/802 [==============================] - 2s 2ms/step - loss: 0.6214 - accuracy: 0.6429
Epoch 8/120
802/802 [==============================] - 2s 3ms/step - loss: 0.6210 - accuracy: 0.6419
Epoch 9/120
802/802 [==============================] - 2s 3ms/step - loss: 0.6210 - accuracy: 0.6420
Epoch 10/120
802/802 [==============================] - 2s 2ms/step - loss: 0.6204 - accura

In [55]:
# Export our model to HDF5 file
nn.save("/content/gdrive/MyDrive/AlphabetSoupCharity4.h5")